In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import scipy.stats as ss
import copy

In [ ]:
flip = lambda y,x: x*y+(1-x)*(1-y)

def cond_gen(x, alphas):
    if type(alphas) == np.ndarray:
        shape = alphas.shape
    else:
        shape = 1
    Z = np.random.uniform(size=shape)**(1/alphas)
    Y = flip(Z, x)
    return(Y)

cond_ltail = lambda y,alpha,x: flip(flip(y,x)**alpha, x)
ltail = lambda y,alpha,p: p*cond_ltail(y,alpha,1)+(1-p)*cond_ltail(y,alpha,0)
preftail = lambda y,alpha,p,pref: flip(ltail(y,alpha,p), 1-pref)

cond_den = lambda y,alpha,x: alpha*flip(y,x)**(alpha-1)

In [ ]:
class env:
    
    def __init__(self, n, alphas, pref, p=0.5):
        alphas = -np.sort(-np.copy(alphas))
        ids = np.arange(n)
        np.random.shuffle(ids)
        self.n = n
        self.sys = pd.DataFrame({
            'p':p, 'rank':np.arange(n),
            'id':ids, 'alpha':alphas,
            'pref':pref})
        self.term = dict()
    
    def obsgen(self):
        newx = np.random.binomial(1,self.sys['p'])
        newy = cond_gen(newx, self.sys['alpha'])
        self.sys['x'] = newx
        self.sys['y'] = newy
        return(self)
    
    def addcol(self, vec, colname):
        self.sys[colname] = np.copy(vec)
        return(self)
    
    def correct(self, subset):
        self.sys.iloc[subset]['report'] = self.sys.iloc[subset]['y']
        return(self)
    
    def newpos(self, subset, n=None):
        if n is None:
            n = self.n
        subset = np.copy(subset)
        m = subset.size
        if m==0:
            return(subset)
        if subset.max()!=n-1:
            return(subset+1)
        temp = self.newpos(subset[:-1], n-1)
        return(np.concatenate((temp, [n-1])))
    
    def rest(self, subset):
        n = self.n
        return(np.arange(n)[[i not in subset for i in range(n)]])
    
    def newrank(self, subset):
        subset = np.copy(subset)
        if subset.size==0:
            return(np.arange(self.n))
        nr = np.arange(self.n)
        npos = self.newpos(subset)
        nr[subset] = npos
        nr[self.rest(subset)] = self.rest(npos)
        return(nr)
    
    def penalize(self, subset):
        nr = self.newrank(subset)
        self.sys['rank'] = np.copy(self.sys['rank'][nr])
        self.sys['alpha'] = np.copy(self.sys['alpha'][nr])
        self.sys = self.sys.sort_values('rank')
        self.addspec()
        return(self)
    
    def addspec(self):
        specloc = (self.sys['pref']*2-1)*(self.n-self.sys['rank'])
        self.sys['spec'] = ss.rankdata(specloc, 'ordinal')
        return(self)
    
    def investigate(self, subset):
        dt = self.sys.iloc[subset]
        dt2 = dt[dt['y']!=dt['report']]
        self.guilty = dt2['rank']
        self.guiltyid = np.copy(dt2.sort_values('id')['id'])
        return(self)

In [ ]:
class nw(nx.Graph):
    
    a=1; k=0.5; pconn=0.5
    
    def tie_env(self, envir):
        self.env = envir
        envir.nw = self
        return(self)
    
    def soc_prod(self,i,j):
        return(self.env.sys.sort_values('id').iloc[[i,j]]['alpha'].prod())
    
    def diff(self,i,j):
        locs = self.env.sys.sort_values('id').iloc[[i,j]]['spec']
        return(np.abs(locs.iloc[0]-locs.iloc[1]))
    
    def shift(self,i,j):
        if i>=j:
            return(self)
        socp = self.soc_prod(i,j)
        d = self.diff(i,j)
        pconn = nw.pconn
        pbreak = pconn/((socp**nw.a)*np.exp(-nw.k*d))
        if (i,j) in self.edges:
            if np.random.binomial(1, pbreak):
                self.remove_edge(i,j)
        else:
            if np.random.binomial(1, pconn):
                self.add_edge(i,j)
        return(self)
    
    def timestep(self):
        for i in self.nodes:
            for j in self.nodes:
                self.shift(i,j)
        return(self)
    
    def changepar(a=a, k=k, pconn=pconn):
        nw.a, nw.k, nw.pconn = a, k, pconn
        return

In [ ]:
class prior(pd.DataFrame):
    
    stdgrid = np.arange(1,200,2)/200
    stdval = 1
    
    def fill(self, p=stdgrid, f=stdval):
        self['p']=p; self['f']=f
        return(self)
    
    def integrate(self):
        return(np.mean(self['f']))
    
    def pnormal(self):
        self['f'] = self['f']/self.integrate()
        return(self)
    
    def bayes(self,another):
        output = prior().fill(self['p'], self['f']*another['f'])
        return(output.pnormal())

In [ ]:
class spvs:
    
    stdgrid = np.arange(1,200,2)/200
    
    def __init__(self, prior=prior().fill(), suspicion=0.05):
        self.prior = prior
        self.s = suspicion
    
    def tie_env(self, envir):
        self.env = envir
        envir.spvs = self
        return(self)
    
    def bias(self):
        self.b = np.mean(self.prior['f']*self.prior['p'])
        return(self.b)
    
    def first_est(self, subset):
        self.bias()
        dt = self.env.sys.iloc[subset]
        calratio = lambda row:cond_den(row['report'],row['alpha'],x=1)/cond_den(row['report'],row['alpha'],x=0)
        ratios = dt.apply(calratio, axis=1)
        ratio = ratios.prod()*self.b/(1-self.b)
        return(ratio/(1+ratio))
    
    def suspects(self, subset, q):
        dt = self.env.sys.iloc[subset].copy()
        dt['bool'] = dt.apply(lambda row:preftail(row['report'],row['alpha'],q,row['pref'])<self.s, axis=1)
        return(dt['rank'][dt['bool']])
    
    def decide(self):
        calratio = lambda row:cond_den(row['report'],row['alpha'],x=1)/cond_den(row['report'],row['alpha'],x=0)
        ratios = self.env.sys.apply(calratio, axis=1)
        ratio = ratios.prod()*self.b/(1-self.b)
        return(1 if ratio>1 else 0)
    
    def condist(prob, dt, colname='report'):
        part1 = dt.apply(lambda row:cond_den(row[colname],row['alpha'],x=1), axis=1).prod()*prob
        part2 = dt.apply(lambda row:cond_den(row[colname],row['alpha'],x=0), axis=1).prod()*(1-prob)
        return(part1+part2)
    
    def update(self):
        temp = prior().fill()
        temp['f'] = temp.apply(lambda row:spvs.condist(row['p'],self.env.sys),axis=1)
        self.prior = self.prior.bayes(temp)
        return(self)
    
    def analysis(self):
        n = self.env.n
        test = np.sort(np.random.choice(range(n),int(n/2),False))
        trust = np.arange(n)[[i not in test for i in range(n)]]
        q = self.first_est(trust)
        suspects = self.suspects(test, q)
        self.env.investigate(suspects)
        self.env.correct(suspects)
        self.env.decision = self.decide()
        self.update()
        return(self)
        

In [ ]:
class term:
    
    stdgrid = np.arange(1,200,2)/200
    
    def __init__(self, pid, prior=prior().fill(), sushat=0.05):
        self.id = pid
        self.prior = prior
        self.sushat = sushat
    
    def tie_env(self, envir):
        self.env = envir
        self.nw = envir.nw
        envir.term[self.id] = self
        return(self)
    
    def bias(self):
        self.b = np.mean(self.prior['f']*self.prior['p'])
        return(self.b)
    
    def getinfo(self):
        dt = self.env.sys
        self.myinfo = dt[dt['id']==self.id]
        neighbors = list(nx.neighbors(self.nw, self.id))
        self.avinfo = dt.sort_values('id').iloc[neighbors]
        self.info = pd.concat([self.myinfo, self.avinfo])
        return(self)
    
    def condist(prob, dt, colname='y'):
        part1 = dt.apply(lambda row:cond_den(row[colname],row['alpha'],x=1), axis=1).prod()*prob
        part2 = dt.apply(lambda row:cond_den(row[colname],row['alpha'],x=0), axis=1).prod()*(1-prob)
        return(part1+part2)
    
    def report(self):
        q = self.bias()
        self.getinfo()
        pref = self.myinfo['pref'].iloc[0]
        y = self.myinfo['y'].iloc[0]
        calratio = lambda row:cond_den(row['y'],row['alpha'],x=1)/cond_den(row['y'],row['alpha'],x=0)
        ratio = self.info.apply(calratio, axis=1).prod()*q/(1-q)
        newq = ratio/(ratio+1)
        caltail = lambda y: preftail(y, self.myinfo['alpha'], newq, pref)
        tails = pd.Series(term.stdgrid).apply(caltail).iloc[:,0]
        safe = term.stdgrid[pd.Series(tails)>self.sushat]
        if safe.size==0:
            return(y)
        elif pref==1:
            return(max(y, safe.max()))
        else:
            return(min(y, safe.min()))
        
    def updateprior(self):
        temp = pd.Series(prior.stdgrid).apply(lambda p: term.condist(p, self.info))
        self.prior = self.prior.bayes(pd.DataFrame({'f':temp}))
        return(self)
    
    def gennewsus(self):
        i,s = self.id, self.sushat
        punid = self.env.guiltyid
        punflag = (i in punid)
        avid = self.avinfo['id']
        avpun = avid[avid.apply(lambda i:(i in punid))]
        jitter = lambda x:x*np.random.uniform(0.9, 1.1)
        if avpun.size == 0:
            self.newsus = jitter(1.1*s) if punflag else jitter(s)
            return
        estimates = [self.env.term[i].sushat for i in avpun]
        maxest = max(estimates)
        self.newsus = jitter(maxest) if punflag else jitter((s+min(s,maxest))/2)
        return
    
    def updatesus(self):
        self.sushat = self.newsus
        return(self)

In [ ]:
def initialize(n, npref, p=0.5):
    envir = env(n, np.arange(2,3,1/n), np.random.choice([1]*npref+[0]*(n-npref), n, False), p=p)
    envir.addcol(np.random.choice(np.arange(0.5,2,1/n),n,False), 'soc')
    envir.addspec()
    net = nw()
    net.add_nodes_from(range(n))
    net.tie_env(envir)
    supervisor = spvs()
    supervisor.tie_env(envir)
    for i in range(n):
        terminal = term(i)
        terminal.tie_env(envir)
    return(envir)

def cycle(envir, log):
    envir.nw.timestep()
    envir.obsgen()
    envir.repo = [envir.term[i].report() for i in envir.sys['id']]
    envir.addcol(envir.repo, 'report')
    envir.spvs.analysis()
    envir.penalize(envir.guilty)
    for i in range(n):
        envir.term[i].updateprior()
        envir.term[i].gennewsus()
    for i in range(n):
        envir.term[i].updatesus()
    log.append(copy.deepcopy(envir))
    return

You can try running the model in the following way:

In [ ]:
n,m = 9,5
envir = initialize(n,m)
nw.changepar(pconn=0.5/n, k=1/n, a=1)
log = [copy.deepcopy(envir)]
for i in range(10):
    cycle(envir, log)

And you may then look into the log for further analysis. The following cell shows how more data can be generated altogether and saved for later analysis.

In [ ]:
import pickle

n=9
wholelog = []

for npref in np.arange(1,9):
    envir = initialize(n,npref)
    nw.changepar(pconn=0.5/n, k=1/n, a=1)
    log = [copy.deepcopy(envir)]
    for i in range(50):
        cycle(envir, log)
    wholelog.append(log)
    
filename = open(''.join(['one_instances.obj']),'wb')
pickle.dump(wholelog, filename)
filename.close()

The code does not work well on Flux (though it is fine on my laptop, maybe there are some module issues on Flux), so I also wrote an R implementation for Flux comoputation.